rpc groups?


In [ ]:
var FUNCTION_GROUPS = ['Function'];
var SELENIUM_GROUPS = ['Selenium'];
var UNITTEST_GROUPS = ['Unit Test']; // TODO: identified using describe
var DEFAULT_GROUPS = ['Available']
var PUBLIC = {
    // rpc basics
    'rpc.ipynb[permissions]' : ['Function', 'Public'],
    'code analysis.ipynb[parameter names]' : ['Function', 'Public'],
    
    // tags on contacts used for permissions groups
    'google contacts.ipynb[google contact settings]': ['Function', 'Inner Circle', 'Public'],

    // searching
    'gulp.ipynb[search notebooks]' : ['Function', 'Public'],
    'interpret questions.ipynb[search notebook questions]' : ['Function', 'Public'],
    
    'data collection.ipynb[meta search all]' : ['Function'], // no Public, use schedule instead
    'data collection.ipynb[schedule search all]' : ['Function', 'Public', 'scheduler'],
    'data collection.ipynb[crawl domain]' : ['Function'], // no Public, use schedule instead
    'data collection.ipynb[schedule crawl domain]': ['Function', 'Public', 'scheduler'],
    
    // date tools
    'data collection.ipynb[tell joke]' : ['Function', 'Public'],
    'google timeline.ipynb[search timeline events]' : ['Function', 'Public', 'map'],
    'google scheduling.ipynb[search calendar]': ['Function', 'Public', 'history'],
    'google scheduling.ipynb[calendar search heatmap]' : ['Function', 'Public', 'slider'],
    
    // dev tools
    'aws.ipynb[latest s3 bucket]' : ['Function', 'Public', 'gallery'],
    'git.ipynb[tip git tree]' : ['Function', 'Public', 'slider'],
    'git.ipynb[megamind]' : ['Function', 'Public', 'editor'],
    'git.ipynb[github commit html acl]' : ['Function', 'Public', 'editor'],
    'child process.ipynb[spawn child process]' : ['Function', 'Public'],
    'twilio.ipynb[accept incoming twilio message]': ['Function', 'Public'],
    'twilio.ipynb[twilio reminder]': ['Function', 'Public', 'scheduler'],
    
    // marketing site
    'marketing scripts.ipynb[contact us handler]': ['Function', 'Public'],
    'convert spreadsheet.ipynb[get sheet identifier]': ['Function', 'Public'],
    'convert spreadsheet.ipynb[sheet marketing import]': ['Function', 'Public'],
    'convert spreadsheet.ipynb[setup sheet backend]': ['Function', 'Public'],
    'convert spreadsheet.ipynb[create a sheet]': ['Function', 'Public'],
    
    // study sauce
    'study sauce.ipynb[authorize app to read profile info]': ['Function', 'Public'],
    'study sauce.ipynb[create a study sauce user directory]': ['Function', 'Public'],
    'study sauce.ipynb[render study sauce cards page]': ['Function', 'Public'],
    'study sauce.ipynb[create a copy of study sauce template]': ['Function', 'Public'],
    
};

module.exports = {
    FUNCTION_GROUPS,
    SELENIUM_GROUPS,
    UNITTEST_GROUPS,
    DEFAULT_GROUPS,
    PUBLIC
}


get cell rpc groups?


In [ ]:
var importer = require('../Core');
var addAST = importer.import('get all ast properties')

// TODO: filter RPC functions by fully unit tested or unlinted?
// TODO: filter by local system groups?
// TODO: move these classifications to import notebook?
var {FUNCTION_GROUPS, SELENIUM_GROUPS,
     UNITTEST_GROUPS, DEFAULT_GROUPS, PUBLIC} = importer.import('rpc groups')

function getUnmatched(cell) {
    try {
        return !cell.questions[0]
            || cell.id != importer.interpret(cell.id2).id
            || cell.id != importer.interpret(cell.questions[0]).id
    } catch (e) {
        return false;
    }
}

function getCellGroups(cell) {
    if(cell.language === 'javascript') addAST(cell);
    return  [].concat.apply([], [
        (cell.groups || []).filter(g => g !== 'Unmatched' && g !== 'Exact'
                                   && g !== 'Corrected' && g !== 'Available'
                                   && g !== 'Error' && g !== 'Uncallable'),
        cell.ast ? [] : ['Error'],
        cell.ast && cell.exports[0] ? FUNCTION_GROUPS : ['Uncallable'],
        cell.filename.includes('Selenium') ? SELENIUM_GROUPS : [],
        cell.questions[0].includes('test') ? UNITTEST_GROUPS : [],
        // TODO: add AST check for describe function call
        [cell.language],
        // filter RPC functions by module.exports
        cell.exports || [],
        getUnmatched(cell) ? ['Unmatched'] : DEFAULT_GROUPS,
        !cell.original || cell.id2 === cell.original ? ['Exact'] : ['Corrected']
    ]).filter((g, i, arr) => arr.indexOf(g) === i)
}

module.exports = getCellGroups;


rpc permissions?


In [ ]:
var _ = require('lodash');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var listInProject = importer.import('list project files');
var getCellGroups = importer.import();
var {PUBLIC} = importer.import('rpc groups')

// find the shortest words from the query to match the same cell
var id2 = (cell) => path.basename(cell.filename) + '[' + cell.questions[0] + ']';

function getPermissions(search) {
    const AVAILABLE = typeof getCachedPermissions !== 'undefined'
        ? getCachedPermissions()
        : []
    
    var public = Object.keys(PUBLIC);
    const PUBLIC_FILTERED = importer.interpret(public)
        .map((cell, i) => Object.assign(cell, {
            id2: id2(cell),
            original: public[i],
            groups: PUBLIC[public[i]]
        }))
        .reduce((acc, cell) => (acc[cell.id2] = groups(cell), acc), {})
    
    if(search) {
        if(typeof search === 'string') {
            search = [search];
        }
        return importer.interpret(search)
            .map((cell, i) => Object.assign(cell, {
                id2: id2(cell),
                original: search[i],
                groups: ['Search']
                    .concat(PUBLIC_FILTERED[id2(cell)] || [])
                    .concat(PUBLIC[id2(cell)] || [])
                    .concat(AVAILABLE[id2(cell)] || [])
            }))
            .reduce((acc, cell) => (acc[cell.id2] = groups(cell), acc), {})
    }
    // Always return the permission set so it can't be manipulated by another command.
    return Object.assign({}, AVAILABLE, PUBLIC_FILTERED);
}

module.exports = getPermissions;


test get permissions?

In [ ]:
var importer = require('../Core');
var getPermissions = importer.import('rpc permissions');

function groupPermissions(groups) {
    if(typeof groups === 'string') groups = [groups];
    var permissions = getPermissions(groups.includes('Available')
                                     ? importer.interpret().map(i => i.id)
                                     : '');
    if(typeof groups !== 'undefined') {
        return Object.keys(permissions)
            .reduce((obj, p) => groups.filter(g => permissions[p]
                    .includes(g)).length > 0 ? (obj[p] = permissions[p], obj) : obj, {});
    }
    return Object.keys(permissions).reduce((types, key) => {
        return permissions[key].reduce((types, group) => {
            if(typeof types[group] === 'undefined') types[group] = {};
            types[group][key] = permissions[key];
        }, types)
    }, {});
}

module.exports = groupPermissions;



filter command permission?


In [ ]:
var importer = require('../Core');
var getPermissions = importer.import('rpc permissions');
var getDaysEvents = importer.import('days events');
var getSettings = importer.import('google contact settings');
var getOauthClient = importer.import('import google calendar api');

var options = {
    calendarId: 'Commands'
};

var alreadyRun = (date, id) => getDaysEvents(new Date(date), options)
    .then(events => events
          .filter(e => e.event.summary.indexOf('Result:') > -1
                  && e.event.summary.indexOf(id) > -1).length > 0);

// TODO: move this logic out to a higher level coordinator?
function filterCommand(command, date, id, user) {
    const props = {};
    return authorizeCalendar(options)
        .then(() => getSettings(user))
        .then(settings => {
            // assign user controls and interpreted command
            try {
                Object.assign(props, settings, {
                    result: importer.interpret(command)
                });
                // if looking for settings use the user name provided to this function for lookup
                // TODO: accept parameters from message context
                if(props.result.id === importer.interpret('google contacts.ipynb[google contact settings]').id) {
                    props.result.runInNewContext = () => settings;
                }
            } catch (e) {
                if((e + '').indexOf('Nothing found') > -1) {
                    Object.assign(props, {result: null});
                } else {
                    throw e;
                }
            }
        })
        .then(() => alreadyRun(date, id))
        .catch(e => console.log(e))
        .then(already => Object.assign(props, {already: already || null}))
}
module.exports = filterCommand;


store rpc result?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var createNewEvent = importer.import('create new calendar event');
var ISODateString = importer.import('convert date iso');
var getOauthClient = importer.import('import google calendar api');
var getResult = importer.import('rpc result');

var options = {
    calendarId: 'Commands'
};

function updateResultEvent(response, executed, isError, isStarting = false) {
    const config = {
        start: {
            dateTime: ISODateString(new Date(executed.date.getTime()))
        },
        end: {
            dateTime: ISODateString(new Date(executed.date.getTime() + 60 * 30 * 1000))
        },
        summary: 'Result: ' + executed.id,
        description: JSON.stringify(response, null, 4).substr(0, 1000),
        colorId: isStarting ? 9 : (isError ? 11 : 10)
    }
    assert(config.colorId !== 10 || !isError,
           'something went wrong with reporting the error ' + JSON.stringify(response, null, 4));
    return createNewEvent(config, options).then(() => response);
}

function storeResult(executed, calendar) {
    if(calendar) {
        options.calendarId = calendar || options.calendarId;
    }
    if(typeof executed === 'undefined' || executed === null
       || executed.already !== false) {
        // skip commands that have already been run
        throw new Error('Nothing to do!');
    }
    
    console.log('creating rpc event ' + JSON.stringify(Object.keys(executed).reduce((acc, k) => {
        acc[k] = (JSON.stringify(executed[k]) || '').substr(0, 200);
        return acc;
    }, {})));
    
    assert(executed.date, 'There should always be a date associated with the event result.');
    
    var isError = false;
    return getOauthClient(options)
        // create a new events to store the results
        .then(() => updateResultEvent({
            time: new Date(),
            result: executed.result.filename,
            command: executed.command,
            parameters: executed.body,
            status: 'starting'
        }, executed, false, true))
        // process the command, this should return a function to be called after event
        .then(() => getResult(executed))
        .catch(e => {
            isError = true;
            const resultError = Object.getOwnPropertyNames(e).reduce((alt, key) => {
                alt[key] = e[key];
                return alt;
            }, {});
            console.log(resultError);
            return resultError;
        })
        // update event with logged results or tracking
        .then(response => updateResultEvent(response, executed, isError || !response))
        .catch(e => console.log(e))
}
module.exports = storeResult;



rpc result?


In [ ]:
var path = require('path');
var assert = require('assert');
var importer = require('../Core');
var groupPermissions = importer.import('test get permissions');
var getParameters = importer.import('function parameters');

function getResult(props) {
    if (props.result === null) {
        throw new Error('command not found, please specify a command');
    }
    assert(props.result.id, 'something is terribly wrong with this execution: ' + JSON.stringify(props.result));
    
    // filter permissions compare id with permissions
    props.allowed = Object.keys(groupPermissions(props.circles || ['Public'])).includes(props.result.id);
    if(!props.allowed) {
        throw new Error('Would have run "' + props.result.id
                        + '" but you don\'t have permission. '
                        + 'Deploy your own server to get access to all RPC functions.');
    }
    
    console.log(`running ${props.command} - ${props.result.id}`)    
    // TODO: make this nicer, ugly because importer doesn't conform to the same importing
    //   style and therefore functions are missing from the context when loaded separately.
    // This is maybe a sign there is something wrong with this style of dependency injection
    var commandResult = props.result.runInNewContext();
    if(commandResult && typeof commandResult[Object.keys(commandResult)[0]] === 'function') {
        commandResult = commandResult[Object.keys(commandResult)[0]];
    }
       
    if(typeof commandResult === 'function') {
        const parameters = getParameters(commandResult);
        var parameterValues;
        if(props.body) {
            if(typeof props.body[0] !== 'undefined') {
                parameterValues = props.body;
            } else {
                parameterValues = parameters.slice(1).map((k, i) => {
                    const p = props.body[k] || props.body[i];
                    if(typeof p === 'undefined' || p === 'undefined') {
                        return;
                    }
                    return p;
                });
            }
        } else {
            parameterValues = [];
        }
        return commandResult.apply(null, parameterValues);
    }
    return commandResult;
}

module.exports = getResult;



get environment?

get environment variables?


In [ ]:
var process = require('process');
var path = require('path');
var Mustache = require('mustache');

var ENVIRONMENT = (key) => ({
    'DEFAULT': {
        'PROFILE_PATH': '{{#HOME}}{{.}}{{/HOME}}{{^HOME}}{{#HOMEPATH}}{{.}}{{/HOMEPATH}}{{^HOMEPATH}}{{USERPROFILE}}{{/HOMEPATH}}{{/HOME}}',
    },
    'BRIAN_RESUME': {
        'CONTACT_DOCID': '1F07qFwP2LO14dJkExjaMHDfuNcB8HfuBQJWiVlmzrAQ',
        'CONTACT_REDIRECT': 'https://briancullinan.com'
    },
    'ILLUMINATI': {
        'CONTACT_DOCID': '16b1Z0sQkYhtMUfP7qMRL3vRBPWQsRbSlntURkMqWGX0',
        'CONTACT_REDIRECT': 'https://www.shopilluminati.com/contact'
    },
    'GOOGLE_RPC': {
        'HOME': '',
        'HOMEPATH': '',
        'USERPROFILE': '',
        'PROFILE_PATH': ''
    },
    'STUDY_REMOTE': {
        'DOCID': '1EGwxT6InTXuvpLujnwKV3asEFDZhhZk2LdosjW2Tz_M',
        'SECRETS_PATH': './client_secret.json',
        'FUNCTIONS_URL': 'https://us-central1-sheet-to-web.cloudfunctions.net/studyRPC',
        'AUTH_REDIRECT': '{{FUNCTIONS_URL}}?function=receiveCode',
        'DOMAIN': 'https://studysauce.sheet-to-web.com',
        'BUCKET': 'studysauce.sheet-to-web.com',
        'GOOGLE_STORAGE_CREDENTIALS': './spahaha-ea443-a78ab2269985.json',
        'GOOGLE_STORAGE_PROJECT': 'spahaha-ea443'
    },
    'STUDY_LOCAL': {
        'DOCID': '1EGwxT6InTXuvpLujnwKV3asEFDZhhZk2LdosjW2Tz_M',
        'SECRETS_PATH': '{{PROFILE_PATH}}./client_secret.json'),
        'FUNCTIONS_URL': 'http://localhost:8010/sheet-to-web/us-central1/studyRPC',
        'AUTH_REDIRECT': '{{FUNCTIONS_URL}}?function=receiveCode',
        'DOMAIN': 'http://localhost:8080',
        'BUCKET': 'studysauce.sheet-to-web.com',
        'GOOGLE_STORAGE_CREDENTIALS': '{{PROFILE_PATH}}.credentials/spahaha-ea443-a78ab2269985.json',
        'GOOGLE_STORAGE_PROJECT': 'spahaha-ea443',
        'PROJECT_OUTPUT': './.output/'
    }
}[key] || {});

var assign = (key) => {
    var envs = Object.assign.apply(null, [
        {
            'ENVIRONMENT': key
        },
        ENVIRONMENT('DEFAULT'),
        ENVIRONMENT(key) || {}
    ])
    var env = Object.keys(envs).reduce((env, e) => {
        env[e.toUpperCase()] = Mustache.render(envs[e], Object.assign({}, process.env, envs));
        return env;
    }, {})
    return Object.assign(process.env, env);
}

function getEnvironment(environment) {
    if(process.env.ENVIRONMENT) {
        assign(process.env.ENVIRONMENT);
    }
    if(environment) {
        assign(environment);
    }
}

module.exports = getEnvironment;

